In [18]:
import os
import shutil
from glob import glob

from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

from albumentations import *
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchsummary import summary
from efficientnet_pytorch import EfficientNet

In [76]:
# define argumentation function
def get_transforms(need=('train', 'val'), img_size=(512, 384), mean=(0.560, 0.524, 0.501), std=(0.233, 0.243, 0.246)):
    transformations = {}
    if 'train' in need:
        transformations['train'] = Compose([
            Resize(img_size[0], img_size[1], p=1.0),
            HorizontalFlip(p=0.5),
            ShiftScaleRotate(p=0.7),
            RandomBrightnessContrast(brightness_limit=(-0.1, 0.3), contrast_limit=(-0.1, 0.3), p=0.5),
            GaussNoise(p=0.7),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0)
        ], p=1.0)
    if 'val' in need:
        transformations['val'] = Compose([
            Resize(img_size[0], img_size[1], p=1.0),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0)
        ], p=1.0)
    return transformations

def tta_transforms(need=('origin', 'flip', 'rotate', 'contrast'), img_size=(512, 384), mean=(0.560, 0.524, 0.501), std=(0.233, 0.243, 0.246)):
    transformations = {}
    if 'origin' in need:
        transformations['origin'] = Compose([
            Resize(img_size[0], img_size[1], p=1.0),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0)
        ], p=1.0)
    if 'flip' in need:
        transformations['flip'] = Compose([
            Resize(img_size[0], img_size[1], p=1.0),
            HorizontalFlip(p=1.0),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0)
        ], p=1.0)
    if 'rotate' in need:
        transformations['rotate'] = Compose([
            Resize(img_size[0], img_size[1], p=1.0),
            ShiftScaleRotate(p=1.0),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0)
        ], p=1.0)
    if 'contrast' in need:
        transformations['contrast'] = Compose([
            Resize(img_size[0], img_size[1], p=1.0),
            RandomBrightnessContrast(brightness_limit=(-0.1, 0.3), contrast_limit=(-0.1, 0.3), p=1.0),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0)
        ], p=1.0)
    return transformations

# define testdataset
class TestDataset(Dataset):
    def __init__(self, path, transform):
        self.path = path
        self.transform = transform
    
    def __getitem__(self, index):
        image = Image.open(self.path[index])
        
        if self.transform:
            image = self.transform(image=np.array(image))
            
        return image['image']

    def __len__(self):
        return len(self.path)

# inference

In [67]:
# load model
title = 'effnetb3_celoss_nokfold_agefilter57'
model_name = '07_0.646_0.632_0.947'  # ex) 14_0.212_0.926_0.200_0.933.


model_path = f'models/{title}/{model_name}.pth'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model_type = title.split('_')[0]
if model_type == 'effnetb3':
    model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=18)
elif model_type == 'resnet50':
    model = models.resnet50(pretrained=True)
    model.fc = nn.Linear(in_features=2048, out_features=18, bias=True)
elif model_type == 'resnet101':
    model = models.resnet101(pretrained=True)
    model.fc = nn.Linear(in_features=2048, out_features=18, bias=True)

model.load_state_dict(torch.load(model_path))
model.to(device)

# inference
submission = pd.read_csv('input/data/eval/info.csv')
test_img_dir = 'input/data/eval/images'

test_path = [f'{test_img_dir}/{img_id}' for img_id in submission.ImageID]


transform = get_transforms()
test_dataset = TestDataset(test_path, transform['val'])

test_loader = DataLoader(
    test_dataset,
    batch_size = 64,
    shuffle = False
)

model.eval()
test_predictions = []
for img in tqdm(test_loader):
    with torch.no_grad():
        img = img.to(device)
        pred = model(img)
        pred_class = torch.max(pred, 1)[1]
        test_predictions.extend(list(pred_class.cpu().numpy()))

submission['ans'] = test_predictions

# save submission
if not os.path.isdir('submissions'):
    os.makedirs('submissions')

if not os.path.isdir(f'submissions/{title}'):
    os.makedirs(f'submissions/{title}')    
    
submission.to_csv(f'submissions/{title}/{model_name}.csv', index=False)
print('* inference, model save done \n')

Loaded pretrained weights for efficientnet-b3


100%|██████████| 197/197 [02:46<00:00,  1.18it/s]

* inference, model save done 



# Inference for ensemble

In [63]:
fold_ensemble = {}

In [64]:
# ensemble inference
titles = ['effnetb3_celoss_nokfold_agefilter',
          'effnetb3_celoss_nokfold_agefilter_alltrain',
          'resnet101_celoss_nokfold_agefilter',
          'resnet50_celoss_nokfold_agefilter'
         ]
          
loop_model = ['06_0.637_0.634_0.963',
              '06_alltraining',
              '03_0.624_0.612_0.935',
              '06_0.630_0.630_0.949'
             ]

for title, m in zip(titles,loop_model):
    
    model_path = f'models/{title}/{m}.pth'

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    model_name = title.split('_')[0]
    if model_name == 'effnetb3':
        model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=18)
    elif model_name == 'resnet50':
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(in_features=2048, out_features=18, bias=True)
    elif model_name == 'resnet101':
        model = models.resnet101(pretrained=True)
        model.fc = nn.Linear(in_features=2048, out_features=18, bias=True)

    model.load_state_dict(torch.load(model_path))
    model.to(device)

    # inference
    submission = pd.read_csv('input/data/eval/info.csv')
    test_img_dir = 'input/data/eval/images'

    test_path = [f'{test_img_dir}/{img_id}' for img_id in submission.ImageID]


    transform = get_transforms()
    test_dataset = TestDataset(test_path, transform['val'])

    test_loader = DataLoader(
        test_dataset,
        batch_size = 64,
        shuffle = False
    )

    model.eval()
    test_percentage = []
#     test_predictions = []
    for img in tqdm(test_loader):
        with torch.no_grad():
            img = img.to(device)
            pred = model(img)
            pred_class = torch.max(pred, 1)[1]
            test_percentage.extend(list(pred.cpu().numpy()))
#             test_predictions.extend(list(pred_class.cpu().numpy()))
    
    fold_ensemble[f'{title}/{m}'] = test_percentage
#     submission['ans'] = test_predictions

#     # save submission
#     if not os.path.isdir('submissions'):
#         os.makedirs('submissions')

#     if not os.path.isdir(f'submissions/{title}'):
#         os.makedirs(f'submissions/{title}')    

#     submission.to_csv(f'submissions/{title}/{i}.csv', index=False)
#     print('* inference, model save done \n')

  0%|          | 0/197 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b3


100%|██████████| 197/197 [03:09<00:00,  1.04it/s]


Loaded pretrained weights for efficientnet-b3


100%|██████████| 197/197 [03:18<00:00,  1.01s/it]


In [55]:
print('* final top models')
for m in fold_ensemble.keys():
    print(m)

* final top models
effnetb3_celoss_nokfold/05_0.595_0.592_0.965
effnetb3_celoss_nokfold/12_0.602_0.599_0.979
effnetb3_celoss_nokfold/14_0.604_0.599_0.964
effnetb3_celoss_nokfold/03_alltraining
effnetb3_celoss_nokfold/02_alltraining
resnet50_notune/25_0.071_0.975_0.064_0.981
resnet50_notune/39_0.053_0.983_0.050_0.989
resnet50_focalloss_nokfold/17_0.579_0.576_0.977
resnet101_focalloss_nokfold/12_0.563_0.579_0.983


In [65]:
# ensemble submission save
ensemble_models = fold_ensemble.keys()
sum_pred = np.zeros((12600,18))
for model in ensemble_models:
    sum_pred += np.array(fold_ensemble[model])

submission['ans'] = np.argmax(sum_pred,axis=1)

if not os.path.isdir('submissions/final_ensemble'):
    os.mkdir('submissions/final_ensemble')
submission.to_csv(f'submissions/final_ensemble/all_ensemble_agefilter58.csv', index=False)

# Inference for TTA

In [73]:
transform

In [77]:
# load model
title = 'effnetb3_celoss_nokfold_agefilter58_aug'
model_name = '10_0.610_0.618_0.969'  # ex) 14_0.212_0.926_0.200_0.933.


model_path = f'models/{title}/{model_name}.pth'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model_type = title.split('_')[0]
if model_type == 'effnetb3':
    model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=18)
elif model_type == 'resnet50':
    model = models.resnet50(pretrained=True)
    model.fc = nn.Linear(in_features=2048, out_features=18, bias=True)
elif model_type == 'resnet101':
    model = models.resnet101(pretrained=True)
    model.fc = nn.Linear(in_features=2048, out_features=18, bias=True)

model.load_state_dict(torch.load(model_path))
model.to(device)

# inference
submission = pd.read_csv('input/data/eval/info.csv')
test_img_dir = 'input/data/eval/images'

test_path = [f'{test_img_dir}/{img_id}' for img_id in submission.ImageID]


transform = tta_transforms()
test_datasets = [TestDataset(test_path, transform['origin']),
                 TestDataset(test_path, transform['flip']),
                 TestDataset(test_path, transform['rotate']),
                 TestDataset(test_path, transform['contrast'])]

tta_ensemble = []
for test_dataset in test_datasets:
    test_loader = DataLoader(
        test_dataset,
        batch_size = 64,
        shuffle = False
    )

    model.eval()
    test_percentage = []
    for img in tqdm(test_loader):
        with torch.no_grad():
            img = img.to(device)
            pred = model(img)
            test_percentage.extend(list(pred.cpu().numpy()))

    tta_ensemble.append(test_percentage)

# save submission
if not os.path.isdir('submissions'):
    os.makedirs('submissions')

if not os.path.isdir(f'submissions/{title}'):
    os.makedirs(f'submissions/{title}')    

sum_pred = np.zeros((12600,18))
for pred in tta_ensemble:
    sum_pred += np.array(pred)

submission['ans'] = np.argmax(sum_pred,axis=1)
    
submission.to_csv(f'submissions/{title}/{model_name}_tta.csv', index=False)
print('* inference, model save done \n')

  0%|          | 0/197 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b3


100%|██████████| 197/197 [02:56<00:00,  1.12it/s]

* inference, model save done 

